In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [6]:
def get_list_of_university_towns():
    #'''Returns a DataFrame of towns and the states they are in from the 
    #university_towns.txt list. The format of the DataFrame should be:
    #DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    #columns=["State", "RegionName"]  )
        
    
    #The following cleaning needs to be done:

    #1. For "State", removing characters from "[" to the end.
    #2. For "RegionName", when applicable, removing every character from " (" to the end.
    #3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    data = pd.read_csv('university_towns.txt', sep=r'^.+\["edit"\]' , header=None, engine='python')
    data.columns = ["RegionName"]
        
    data['State'] = data['RegionName'].where(data['RegionName'].str.endswith('[edit]'))
        
    data['State'] = data['State'].fillna(method='ffill')
         
    data['State'] = data['State'].str.replace('\[.*\]','')
    data['RegionName'] = data['RegionName'].str.replace('\[.*\]','')
    data['RegionName'] = data['RegionName'].str.replace('\(.*\)','')
    
    
    data.drop(data[data['RegionName'] == data['State']].index,inplace=True)
    data.reset_index(inplace=True,drop=True)
    
    data['State'] = data['State'].str.strip()
    data['RegionName'] = data['RegionName'].str.strip()
    cols = ['State','RegionName']
    data = data[cols]
    
    data.set_index('State',inplace=True)
    #data.to_records().tolist()
    
    return data

In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp_data = pd.ExcelFile('gdplev.xls')
    gdp_data.sheet_names
    gdp = gdp_data.parse("Sheet1")
    gdp = gdp.dropna(axis=1,how='all')
    gdp = gdp.dropna(axis=0,how='all')    
    gdp.columns = ['Period-A','GDP1-A','GDP2-A','Quarter','GDP1-Q','GDP chained']
    
    gdp.drop(['Period-A','GDP1-A','GDP2-A','GDP1-Q'],axis=1,inplace=True)
    gdp.drop(gdp.index[:3], inplace=True)
    gdp.reset_index(inplace=True,drop=True)
    
    gdp['GDP Change'] = gdp['GDP chained'] - gdp['GDP chained'].shift(1)
    
    gdp['Change'] = gdp['GDP Change'].apply(lambda x: "decline" if (x < 0 ) else "growth" )
    
    s = gdp['Quarter'].where((gdp['Change'] == "decline") & (gdp['Change'].shift(-1) == "decline") & (gdp['Change'].shift(1) == "growth") & gdp['Quarter'].str.startswith("2"))
    
    res = s[~s.isnull()].values[0]
    return res

In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp_data = pd.ExcelFile('gdplev.xls')
    gdp_data.sheet_names
    gdp = gdp_data.parse("Sheet1")
    gdp = gdp.dropna(axis=1,how='all')
    gdp = gdp.dropna(axis=0,how='all')    
    gdp.columns = ['Period-A','GDP1-A','GDP2-A','Quarter','GDP1-Q','GDP chained']
    
    gdp.drop(['Period-A','GDP1-A','GDP2-A','GDP1-Q'],axis=1,inplace=True)
    gdp.drop(gdp.index[:3], inplace=True)
    gdp.reset_index(inplace=True,drop=True)
    
    gdp['GDP Change'] = gdp['GDP chained'] - gdp['GDP chained'].shift(1)
    
    gdp['Change'] = gdp['GDP Change'].apply(lambda x: "decline" if (x < 0 ) else "growth" )
    
    s = gdp['Quarter'].where((gdp['Change'] == "growth") & (gdp['Change'].shift(1) == "growth") & (gdp['Change'].shift(2) == "decline") & (gdp['Change'].shift(3) == "decline") & gdp['Quarter'].str.startswith("2"))
    
    res = s[~s.isnull()].values[0]
    return res

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp_data = pd.ExcelFile('gdplev.xls')
    gdp_data.sheet_names
    gdp = gdp_data.parse("Sheet1")
    gdp = gdp.dropna(axis=1,how='all')
    gdp = gdp.dropna(axis=0,how='all')    
    gdp.columns = ['Period-A','GDP1-A','GDP2-A','Quarter','GDP1-Q','GDP chained']
    
    gdp.drop(['Period-A','GDP1-A','GDP2-A','GDP1-Q'],axis=1,inplace=True)
    gdp.drop(gdp.index[:3], inplace=True)
    gdp.reset_index(inplace=True,drop=True)
    
    gdp['GDP Change'] = gdp['GDP chained'] - gdp['GDP chained'].shift(1)
    
    gdp['Change'] = gdp['GDP Change'].apply(lambda x: "decline" if (x < 0 ) else "growth" )
    
    recession_start = gdp['Quarter'].where((gdp['Change'] == "decline") & (gdp['Change'].shift(-1) == "decline") & (gdp['Change'].shift(1) == "growth") & gdp['Quarter'].str.startswith("2"))
    
    recession_end = gdp['Quarter'].where((gdp['Change'] == "growth") & (gdp['Change'].shift(1) == "growth") & (gdp['Change'].shift(2) == "decline") & (gdp['Change'].shift(3) == "decline") & gdp['Quarter'].str.startswith("2"))
    
  
    
    rstart = recession_start[~recession_start.isnull()]
    rend = recession_end[~recession_end.isnull()]
    
    
    
    gdp.set_index(['Quarter'],inplace=True)
    res_gdp = gdp[str(rstart.iloc[0]):str(rend.iloc[0])]
    
    s = str(res_gdp.loc[res_gdp['GDP chained'] == res_gdp['GDP chained'].min()].index.values[0])
    
    return s

In [10]:
def convert_housing_data_to_quarters():
    
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    data.replace({"State": states},inplace=True)
    data.set_index(['State','RegionName'],drop=True, inplace=True)
    
    for i in range(1997,2000):
        for j in range(1,13):
            if ( j < 10):
                j = '0' + str(j)
            data.drop(str(i) +'-'+ str(j),axis=1,inplace=True)
            
    for j in range(4,13):
        if ( j < 10):
                j = '0' + str(j)
        data.drop('1996' +'-'+ str(j),axis=1,inplace=True)

    for i in range(2000,2016):
        data[str(i)+"q1"] = data[[str(i)+'-'+str('01'),str(i)+'-'+str('02'),str(i)+'-'+str('03')]].mean(axis=1)
        data[str(i)+"q2"] = data[[str(i)+'-'+str('04'),str(i)+'-'+str('05'),str(i)+'-'+str('06')]].mean(axis=1)
        data[str(i)+"q3"] = data[[str(i)+'-'+str('07'),str(i)+'-'+str('08'),str(i)+'-'+str('09')]].mean(axis=1)
        data[str(i)+"q4"] = data[[str(i)+'-'+str('10'),str(i)+'-'+str('11'),str(i)+'-'+str('12')]].mean(axis=1)
        
    data[str(2016)+"q1"] = data[[str(2016)+'-'+str('01'),str(2016)+'-'+str('02'),str(2016)+'-'+str('03')]].mean()   
    data[str(2016)+"q2"] = data[[str(2016)+'-'+str('04'),str(2016)+'-'+str('05'),str(2016)+'-'+str('06')]].mean()   
    data[str(2016)+"q3"] = data[[str(2016)+'-'+str('07'),str(2016)+'-'+str('08')]].mean()   
    
    for i in range(2000,2016):
        for j in range(1,13):
            if ( j < 10):
                j = '0' + str(j)
            data.drop(str(i) +'-'+ str(j),axis=1,inplace=True)
            
    for j in range(1,9):
        if ( j < 10):
            j = '0' + str(j)
        data.drop(str(2016) +'-'+ str(j),axis=1,inplace=True)   
    data.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1,inplace=True)        
    return data     

In [123]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()
    
    rstart = get_recession_start()
    rend = get_recession_end()
    rbottom = get_recession_bottom()
    df['price ratio'] = df.iloc[:,df.columns.get_loc(rstart) - 1] / df[rbottom]
    
    
    hp_df = df.loc[:,rstart:rend]
    hp_df['Price Ratio'] = df['price ratio']
    
    univ_town = get_list_of_university_towns()
    univ_town.reset_index(inplace=True)
    univ_town.set_index(['State','RegionName'],drop=True,inplace=True)
    
    univ_town_hp_df = pd.merge(hp_df, univ_town, how='inner', left_index=True, right_index=True)
    
    non_univ_town_hp = pd.merge(hp_df,univ_town, how='left', left_index=True, right_index=True,indicator=True)
    non_univ_town_hp_df = non_univ_town_hp.loc[non_univ_town_hp['_merge'] == "left_only"]
    non_univ_town_hp_df.drop(['_merge'],axis=1,inplace=True)
    
    univ_town_hp_df_size = univ_town_hp_df.columns.size
    new_univ_town_hp_df = univ_town_hp_df.iloc[:,0:univ_town_hp_df_size-1]
    
    non_univ_town_hp_df_size = non_univ_town_hp_df.columns.size
    new_non_univ_town_hp_df = non_univ_town_hp_df.iloc[0:,0:non_univ_town_hp_df_size-1]
    
    result = ttest_ind(new_univ_town_hp_df,new_non_univ_town_hp_df,nan_policy='omit')
    
    
    mean_pr_univ_town = univ_town_hp_df['Price Ratio'].mean()
    mean_pr_non_univ_town = non_univ_town_hp_df['Price Ratio'].mean()
    
    result_tuple = (True,result.pvalue,'university town')
    
    return result_tuple

In [124]:
res = run_ttest()
res

C:\Users\Gracy\Anaconda2\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True,
 masked_array(data = [ 0.01379052  0.01669579  0.01852844  0.02140923  0.03148543  0.03530609],
              mask = False,
        fill_value = 1e+20),
 'university town')